In [4]:
import os
import os.path as osp
import sgkit as sg
import numpy as np
from sgkit_plink import read_plink
from sgkit.stats import regenie
rs = np.random.RandomState(0)

In [2]:
path = osp.join(os.environ['WORK_DIR'], 'data/gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1.bed')
path

'/home/jovyan/work/data/gwas/tutorial/1_QC_GWAS/HapMap_3_r3_1.bed'

In [12]:
ds = read_plink(path.replace('.bed', ''))
mask = rs.random_sample(size=ds.dims['variants']) < (100_000/ds.dims['variants'])
ds = ds.sel(variants=mask)
ds

<xarray.Dataset>
Dimensions:             (alleles: 2, ploidy: 2, samples: 165, variants: 100079)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    variant/contig      (variants) int16 0 0 0 0 0 0 0 ... 23 23 23 23 23 23 23
    variant/position    (variants) int32 dask.array<chunksize=(100079,), meta=np.ndarray>
    variant/alleles     (variants, alleles) |S1 dask.array<chunksize=(100079, 1), meta=np.ndarray>
    sample/id           (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    call/genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(14284, 165, 2), meta=np.ndarray>
    call/genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(14284, 165, 2), meta=np.ndarray>
    variant/id          (variants) <U1 dask.array<chunksize=(100079,), meta=np.ndarray>
    sample/family_id    (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/paternal_id  (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/maternal_id  (samples) <U1 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/sex          (samples) int8 dask.array<chunksize=(165,), meta=np.ndarray>
    sample/phenotype    (samples) int8 dask.array<chunksize=(165,), meta=np.ndarray>
Attributes:
    contigs:  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',...

In [13]:
ds['sample/id'].values[:5]

array(['NA06989', 'NA11891', 'NA11843', 'NA12341', 'NA12739'], dtype='<U7')

In [14]:
ds['variant/position'].values[:5]

array([789326, 847691, 930066, 937641, 975763], dtype=int32)

In [15]:
# TODO: mean impute missing
n_sample = ds.dims['samples']
n_covariate = 3
n_trait = 2
ds['call/dosage'] = ds['call/genotype'].sum(dim='ploidy')
ds["sample/covariate"] = (("samples", "covariates"), rs.normal(size=(n_sample, n_covariate)))
ds["sample/trait"] = (("samples", "traits"), rs.normal(size=(n_sample, n_trait)))

In [16]:
np.unique(ds['call/dosage'], return_counts=True)

(array([-2,  0,  1,  2]), array([   43109, 10904474,  4485551,  1079901]))

In [17]:
%%time
dsr = regenie.regenie(ds, dosage='call/dosage', covariates='sample/covariate', traits='sample/trait', variant_block_size=1000)
dsr

CPU times: user 3.86 s, sys: 49.8 ms, total: 3.91 s
Wall time: 3.86 s


<xarray.Dataset>
Dimensions:          (alphas: 5, blocks: 114, contigs: 24, outcomes: 2, samples: 165)
Dimensions without coordinates: alphas, blocks, contigs, outcomes, samples
Data variables:
    base_prediction  (blocks, alphas, samples, outcomes) float64 dask.array<chunksize=(1, 5, 17, 2), meta=np.ndarray>
    meta_prediction  (samples, outcomes) float64 dask.array<chunksize=(17, 2), meta=np.ndarray>
    loco_prediction  (contigs, samples, outcomes) float64 dask.array<chunksize=(1, 17, 2), meta=np.ndarray>

In [18]:
dsr['base_prediction'].data

,Array,Chunk
Bytes,1.50 MB,1.36 kB
Shape,"(114, 5, 165, 2)","(1, 5, 17, 2)"
Count,30703 Tasks,1140 Chunks
Type,float64,numpy.ndarray


In [19]:
%%time
BP = dsr['base_prediction'].data.compute()
BP.shape

(114, 5, 165, 2)

In [20]:
dsr['meta_prediction'].data

dask.array<rechunk-merge, shape=(165, 2), dtype=float64, chunksize=(17, 2), chunktype=numpy.ndarray>

In [21]:
%%time
MP = dsr['meta_prediction'].data.compute()
MP.shape

CPU times: user 12min 42s, sys: 20.1 s, total: 13min 2s
Wall time: 53.6 s


(165, 2)

In [22]:
dsr['loco_prediction'].data

,Array,Chunk
Bytes,63.36 kB,272 B
Shape,"(24, 165, 2)","(1, 17, 2)"
Count,127746 Tasks,240 Chunks
Type,float64,numpy.ndarray


In [24]:
%%time
LP = dsr['loco_prediction'].data.compute()
LP.shape

CPU times: user 12min 41s, sys: 18.7 s, total: 12min 59s
Wall time: 58.5 s


(24, 165, 2)